In [1]:
import numpy as np

# 5.2 連鎖律
- 合成関数の微分の性質
- 高速に勾配を計算する誤差逆伝播法に応用する
- ニューラルネット全体の関数を，各ノードの局所的な関数とその先の様々な関数の合成関数と考えて応用

$$
z = f(t) \\
t = g(x, y)
$$
とすると，次のように合成関数の微分を求めることができる
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

***

### 単純な乗算レイヤ，加算レイヤ

- 単純な乗算レイヤの例．
- 様々な複雑な演算を含む損失関数を $ L(z) $ とする．
- その内，一つの乗算ノードで $ z = xy $ という演算が行われるとする．

このとき，次のように勾配を求めることができる
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial x} = \frac{\partial L}{\partial z} y \\　\\
\frac{\partial L}{\partial y} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial y} = \frac{\partial L}{\partial z} x
$$
- $ \frac{\partial L}{\partial z} $ は乗算ノードの出力 $ z $ での $ L $ の偏微分．
- $ x, y $ を勾配計算に用いるため，順伝播の入力を保持しておく必要がある．

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [6]:
class AddLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout
        dy = dout
        return dx, dy

# 5.5 活性化関数レイヤ

### ReLUレイヤ
#### ReLU
$$
y = 
\begin{cases}
    x \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

#### ReLUの偏微分
$$
\frac{\partial y}{\partial x} = 
\begin{cases}
    1 \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

In [10]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        # 入力が 0 以下のインデックスを保持し，その値は0とする
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        # 順伝播の入力が 0 以下であったインデックスの値は 0 ，そのほかはそのまま逆伝播
        dout[self.mask] = 0
        dx = dout
        
        return dx

### Sigmoidレイヤ
#### Sigmoid
$$
y = \frac{1}{1 + \exp(-x)}
$$

#### Sigmoidの偏微分
$$
t = 1 + \exp(-x)
$$

とおくと，
$$
y = \frac{1}{1 + \exp(-x)} = \frac{1}{t} = t^{-1}
$$

また，
$$
\begin{align}
\frac{\partial y}{\partial t} &= -t^{-2} = -(1 + \exp(-x))^{-2}\\
\frac{\partial t}{\partial x} &= -\exp(-x)
\end{align}
$$

であるから，
$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial y}{\partial t} \frac{\partial t}{\partial x} \\ \\
&= \frac{\exp(-x)}{\{1 + \exp(-x)\}^{2}} \\ \\
&= \frac{\exp(-x)}{1 + \exp(-x)} \frac{1}{1 + \exp(-x)} \\ \\
&= \left(\frac{1 + \exp(-x)}{1 + \exp(-x)} - \frac{1}{1 + \exp(-x)} \right) \frac{1}{1 + \exp(-x)} \\ \\
&= (1 - y) y
\end{align}
$$

In [11]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        # 逆伝播の入力に，Sigmoidの偏微分をかける
        dx = dout * (1.0 - self.out) * self.out
        
        return dx